In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
from functools import reduce

In [16]:
# import csvs

season_1617 = pd.read_csv("cleaned_bball_data/2016_17_nbaavgs.csv")
season_1718 = pd.read_csv("cleaned_bball_data/2017_18_nbaavgs.csv")
season_1819 = pd.read_csv("cleaned_bball_data/2018_19_nbaavgs.csv")
season_1920 = pd.read_csv("cleaned_bball_data/2019_20_nbaavgs.csv")
season_2021 = pd.read_csv("cleaned_bball_data/2020_21_nbaavgs.csv")
season_1920.head()

,ID,Player,G,GS,MP,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
0,1557,Steven Adams,63,63,26.7,4.5,7.6,0.592,0.0,0.0,...,9.3,2.3,0.8,1.1,1.5,1.9,10.9,20.5,2.9,2.1
1,1558,Bam Adebayo,72,72,33.6,6.1,11.0,0.557,0.0,0.2,...,10.2,5.1,1.1,1.3,2.8,2.5,15.9,20.3,3.4,3.3
2,1559,LaMarcus Aldridge,53,53,33.1,7.4,15.0,0.493,1.2,3.0,...,7.4,2.4,0.7,1.6,1.4,2.4,18.9,19.7,1.4,1.5
3,1560,Kyle Alexander,2,0,6.5,0.5,1.0,0.500,0.0,0.0,...,1.5,0.0,0.0,0.0,0.5,0.5,1.0,4.7,-10.1,0.0
4,1561,Nickeil Alexander-Walker,47,1,12.6,2.1,5.7,0.368,1.0,2.8,...,1.8,1.9,0.4,0.2,1.1,1.2,5.7,8.9,-4.6,-0.4


In [27]:
# merge df's into one df

data_frames = [season_1617, season_1718, season_1819, season_1920, season_2021]

seasons_df_merged = pd.concat(data_frames)

seasons_df_merged.head()

,ID,Player,G,GS,MP,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
0,1,Álex Abrines,68,6,15.5,2.0,5.0,0.393,1.4,3.6,...,1.3,0.6,0.5,0.1,0.5,1.7,6.0,10.1,-1.6,0.1
1,2,Quincy Acy,38,1,14.7,1.8,4.5,0.412,1.0,2.4,...,3.0,0.5,0.4,0.4,0.6,1.8,5.8,11.8,-2.1,0.0
2,3,Steven Adams,80,80,29.9,4.7,8.2,0.571,0.0,0.0,...,7.7,1.1,1.1,1.0,1.8,2.4,11.3,16.5,-0.2,1.1
3,4,Arron Afflalo,61,45,25.9,3.0,6.9,0.440,1.0,2.5,...,2.0,1.3,0.3,0.1,0.7,1.7,8.4,8.9,-3.6,-0.7
4,5,Alexis Ajinça,39,15,15.0,2.3,4.6,0.500,0.0,0.1,...,4.5,0.3,0.5,0.6,0.8,2.0,5.3,12.9,-3.3,-0.2


In [24]:
# make "Player" its own df

player = [seasons_df_merged["Player"]]
player_name = pd.concat(player, axis=1)
player_name.head()

,Player
0,Álex Abrines
1,Quincy Acy
2,Steven Adams
3,Arron Afflalo
4,Alexis Ajinça


In [28]:
# remove players from merged df
seasons_df_merged.drop('Player', inplace=True, axis=1)

seasons_df_merged.head()

,ID,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP
0,1,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,...,1.3,0.6,0.5,0.1,0.5,1.7,6.0,10.1,-1.6,0.1
1,2,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,...,3.0,0.5,0.4,0.4,0.6,1.8,5.8,11.8,-2.1,0.0
2,3,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,...,7.7,1.1,1.1,1.0,1.8,2.4,11.3,16.5,-0.2,1.1
3,4,61,45,25.9,3.0,6.9,0.440,1.0,2.5,0.411,...,2.0,1.3,0.3,0.1,0.7,1.7,8.4,8.9,-3.6,-0.7
4,5,39,15,15.0,2.3,4.6,0.500,0.0,0.1,0.000,...,4.5,0.3,0.5,0.6,0.8,2.0,5.3,12.9,-3.3,-0.2


In [29]:
# scale the merged df

nba_scaled = StandardScaler().fit_transform(seasons_df_merged)
print(nba_scaled[0:5])


[[-1.7313911   0.84057258 -0.61076908 -0.439592   -0.52437437 -0.42458376
  -0.45091985  0.62442807  0.5117518   0.60858312 -0.92894692 -0.90533306
  -0.53476297  0.2484454  -0.56296384 -0.65147414  0.90934124 -0.69435493
  -0.94452773 -0.92358072 -0.72956628 -0.29148103 -0.7232388  -0.73439543
   0.01954654 -0.41575204 -0.40671383 -0.01074125 -0.36293347]
 [-1.73007145 -0.36939837 -0.79921158 -0.526396   -0.6141206  -0.53204171
  -0.27835016  0.14212009 -0.05147736  0.80955228 -0.76379008 -0.69875794
  -0.63684952  0.16118468 -0.11909718 -0.10859817  0.20266437 -0.42813398
  -0.13212948 -0.23017264 -0.78580542 -0.53461345  0.01566955 -0.61067653
   0.14750085 -0.44802806 -0.16552948 -0.08451357 -0.44506989]
 [-1.72875179  1.32456097  2.17817999  1.12288002  0.68719967  0.26314712
   1.16578569 -1.06364987 -1.17793568 -1.94372526  1.32819653  1.10139675
   0.61174745  0.59748829  0.47272504  0.85651465 -0.4610389   3.56518015
   0.78858853  1.68689678 -0.44837057  1.16731346  1.4934862

In [30]:
# Initialize PCA model
pca = PCA(n_components=3)

In [31]:
# Get 3 principal components for the iris data.
nba_pca = pca.fit_transform(nba_scaled)

In [32]:
# create PCA df
df_nba_pca = pd.DataFrame(
    data = nba_pca, columns = ["principal component 1", "principal component 2", "principal component 3"])
df_nba_pca.head()

,principal component 1,principal component 2,principal component 3
0,-1.991917,-1.096877,1.548250
1,-1.749033,-0.150640,0.531634
2,3.788961,3.635102,-2.943222
3,-0.628658,-1.057711,0.834913
4,-1.314036,2.148602,-1.542362


In [33]:
pca.explained_variance_ratio_

array([0.49138035, 0.12744592, 0.08857485])